# Imports

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
from pprint import pprint

# Load, Prepare and Split Data

### Load & Prepare Data

In [6]:
# df = pd.read_csv('data/train.csv')
# df_len = len(df)
#df.head()

# Handling missing / incorrect data
# There is no missing or incorrect data, so this will remain empty for now
#df.info() 

In [7]:
# Titanic dataset
df = pd.read_csv("data/Titanic.csv")
df["label"] = df.Survived
df = df.drop(["PassengerId", "Survived", "Name", "Ticket", "Cabin"], axis=1)

# handling missing values
median_age = df.Age.median()
mode_embarked = df.Embarked.mode()[0]

df = df.fillna({"Age": median_age, "Embarked": mode_embarked})

### Split Data

In [8]:
training_data, testing_data = train_test_split(df, test_size=0.20, random_state=100, shuffle=True)
training_data.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,label
408,3,male,21.0,0,0,7.7750,S,0
480,3,male,9.0,5,2,46.9000,S,0
510,3,male,29.0,0,0,7.7500,Q,1
609,1,female,40.0,0,0,153.4625,S,1
547,2,male,28.0,0,0,13.8625,C,1


# Is our data pure?

In [9]:
def check_purity(data):
    
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False

# Classify

In [10]:
def classify_data(data):

    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]

    return classification